# VHH Design

### Case Study: Targeting CCL2 (Chemokine Ligand 2)

This notebook demonstrates the end-to-end computational pipeline for designing VHH (Nanobody) binders. We use a combination of deep learning-based backbone generation, sequence design, and flow-based affinity maturation.

---

## Design Pipeline

The workflow is organized into four key modules:

### Hotspot Identification
* **Objective**: Define the binding epitope on the **CCL2** surface.
* **Selection Criteria**: 
    * Focused on specific **hotspots** critical for protein-protein interactions (PPI).
    * Preference for **hydrophobic patches** to maximize surface complementarity.

### Step.1 Backbone Generation & Sequence Design
* **Backbone**: Sampling diverse CDR configurations that fit the target epitope using **PPIFlow**.
* **Sequence**: Inverse folding and amino acid sequence optimization using **AbMPNN** .

### Step.2 Scoring & Filtering
* **Validation**: High-throughput screening of designs using **FlowPacker** and **AF3Score**.
* **Metrics**: Candidates are filtered based on:
    * **$iPTM$**
    * **$PTM$**
### Step.3 Rosetta interface 
### Step.4 Affinity Maturation (PPIFlow)
* **Refinement**: Top-tier candidates undergo maturation using **PPIFlow (Partial Flow version)**.
* **Optimization**: Leveraging discrete-continuous flow matching to fine-tune the interface residues for sub-nanomolar affinity.

### Step.5 Scoring & Filtering
### Step.6 Rosetta relax 
### Step.7 AF3 refold

In [ ]:
project_dir='.'
demo_scripts='demo_scripts'

# Stes.1.1 Backbone Generation

Run the following command in the ppiflow environment on a compute node.

In [ ]:
import os
from pathlib import Path
import subprocess

def submit_nanobody_job(
    antigen_pdb: str,
    framework_pdb: str,
    specified_hotspots: str,
    samples_per_target: int,
    output_dir: str,
    job_name: str = "PPIFlow_nanobody",
    gpu_partition: str = "gpu41,gpu43,gpu53,gpu51",
    gpus: int = 1,
    cpus: int = 4,
    model_weights: str = "path_to_ckpt",
    config: str = "configs/inference_nanobody.yaml",
    antigen_chain: str = "C",
    heavy_chain: str = "A",
    light_chain: str = "B",
    cdr_length: str = "CDRH1,8-8,CDRH2,8-8,CDRH3,9-21",
    conda_python: str = "miniconda3/envs/ppiflow/bin/python",
    log_dir: str = "logs",
):
    """Generate SLURM shell script for nanobody backbone generation and submit it."""
    
    Path(log_dir).mkdir(exist_ok=True, parents=True)
    Path(output_dir).mkdir(exist_ok=True, parents=True)
    
    sh_content = f"""#!/bin/bash
#SBATCH -p {gpu_partition}
#SBATCH --gres=gpu:{gpus}
#SBATCH --ntasks=1
#SBATCH --cpus-per-task={cpus}
#SBATCH -N 1
#SBATCH -o {log_dir}/%j_%x.out
#SBATCH -J {job_name}
export MPLBACKEND=Agg

echo "Job started at: $(date)"
start_time=$(date +%s)

ANTIGEN_PDB="{antigen_pdb}"
FRAMEWORK_PDB="{framework_pdb}"
SPECIFIED_HOTSPOTS="{specified_hotspots}"
CDR_LENGTH="{cdr_length}"
SAMPLES_PER_TARGET="{samples_per_target}"
MODEL_WEIGHTS="{model_weights}"
OUTPUT_DIR="{output_dir}"
NAME=$(basename $OUTPUT_DIR)

ANTIGEN_CHAIN="{antigen_chain}"
HEAVY_CHAIN="{heavy_chain}"
LIGHT_CHAIN="{light_chain}"
CONFIG="{config}"

{conda_python} sample_antibody_nanobody.py \\
    --antigen_pdb $ANTIGEN_PDB \\
    --framework_pdb $FRAMEWORK_PDB \\
    --antigen_chain $ANTIGEN_CHAIN \\
    --heavy_chain $HEAVY_CHAIN \\
    --specified_hotspots $SPECIFIED_HOTSPOTS \\
    --cdr_length $CDR_LENGTH \\
    --samples_per_target $SAMPLES_PER_TARGET \\
    --config $CONFIG \\
    --model_weights $MODEL_WEIGHTS \\
    --output_dir $OUTPUT_DIR \\
    --name $NAME

end_time=$(date +%s)
echo "Job ended at: $(date)"
elapsed=$((end_time - start_time))
echo "Elapsed time: $elapsed seconds"
"""

    sh_path = Path(output_dir) / f"{job_name}.sh"
    with open(sh_path, "w") as f:
        f.write(sh_content)
    
    print(f"SLURM script written to {sh_path}")

    subprocess.run(["sbatch", str(sh_path)])
    print(f"Job submitted with sbatch.")


submit_nanobody_job(
    antigen_pdb=f"{project_dir}/test/input/4dn4.pdb",
    framework_pdb=f"{project_dir}/test/input/7xl0_nanobody_framework.pdb",
    antigen_chain='M',
    specified_hotspots="M28,M39,M55,M61",
    samples_per_target=10,
    output_dir=f"{project_dir}/test/4dn4_ccl2"
)


If you are working on a compute node, you can directly run the Python code below.

```
python sample_antibody_nanobody.py \
    --antigen_pdb path_to_input/4dn4.pdb \
    --framework_pdb path_to_input/7xl0_nanobody_framework.pdb \
    --antigen_chain M \
    --heavy_chain A \
    --specified_hotspots "M28,M39,M55,M61" \
    --cdr_length "CDRH1,8-8,CDRH2,8-8,CDRH3,9-21" \
    --samples_per_target 10 \
    --config configs/inference_nanobody.yaml \
    --model_weights path_to_ckpt/epoch=122-step=25461.ckpt \
    --output_dir path_to_test/nanobody_test/4dn4_ccl2 \
    --name 4dn4_ccl2
```

## Arguments

To avoid potential runtime errors, all file paths should be specified as **absolute paths**.

- `--antigen_pdb`
Path to the antigen PDB structure.

- `--framework_pdb`
Input nanobody framework structure.

The framework should have all three CDR regions removed according to the `IMGT` numbering scheme.

If you do not have a custom framework, you may use the one provided at `./Framework/`.

- `--antigen_chain`
Chain ID of the antigen in the PDB file.

- `--heavy_chain`
Chain ID of the nanobody (heavy chain).

- `--specified_hotspots`

Antigen hotspot residues used to guide backbone generation.

Residue indices must be consistent with the numbering in the input structure.

- `--cdr_length`
Length ranges for the three CDRs (CDRH1, CDRH2, CDRH3).

- `--samples_per_target`
Number of backbone conformations sampled for each target.

- `--config`
Path to the inference configuration file.

- `--model_weights`
Path to the pretrained model checkpoint.

- `--output_dir`
Directory for saving generated backbone structures.

- `--name`
Prefix used for naming the generated structure files
(e.g., `4dn4_ccl2_0.pdb`).

Lowercase characters are recommended to ensure consistency with
downstream **AF3Score** processing.

In [ ]:
# cdr interface ratio >= 0.6
import pandas as pd
import os,shutil
df=pd.read_csv(f'{project_dir}/test/4dn4_ccl2/sample_metrics.csv')
df_f=df[df['cdr_interface_ratio']>=0.6]
print(df_f.shape)

out_dir = f'{project_dir}/test/4dn4_ccl2/filtered_links'
os.makedirs(out_dir, exist_ok=True)

df_f.apply(lambda row: shutil.copy(os.path.abspath(row['pdb_path']), 
                                  os.path.join(out_dir, row['sample'])), axis=1)

print(f"Linked {len(df_f)} files to {out_dir}")

# Step.1.2 Sequence design


In [ ]:
!python $demo_scripts/make_fix_csv.py test/4dn4_ccl2/filtered_links

In [ ]:
!sbatch $demo_scripts/run_abmpnn.sh test/4dn4_ccl2/filtered_links test/4dn4_ccl2_fa test/4dn4_ccl2/filtered_links/fixed_positions.csv

Submitted batch job 2910054


# Step.2 Scoring & Filtering
The code repository will be updated in subsequent releases. 

For now, we briefly outline the overall logic of the pipeline: 

FlowPacker is first used to add side chains onto the backbones designed by PPIFlow, and the resulting full-atom structures are then evaluated using AF3Score for structural and interface quality assessment.

In [ ]:
import os
import csv
from Bio import SeqIO

def direct_fasta_to_csv(input_dirs: list, output_csv: str, suffix: str = ".pdb"):

    seen_seqs = set()  

    os.makedirs(os.path.dirname(os.path.abspath(output_csv)), exist_ok=True)

    with open(output_csv, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["link_name", "seq", "seq_idx"]) 
        for folder in input_dirs:
            if not os.path.exists(folder): continue
            files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(('.fasta', '.fa'))]

            for file_path in files:
                base_name = os.path.splitext(os.path.basename(file_path))[0]

                for i, record in enumerate(SeqIO.parse(file_path, "fasta")):
                    if i == 0: 
                        continue 

                    seq_str = str(record.seq)
                    if seq_str in seen_seqs: 
                        continue 
                    seen_seqs.add(seq_str)
                    link_name = f"{base_name}{suffix}"
                    seq_idx = str(i)

                    writer.writerow([link_name, seq_str, seq_idx])

    print(f"✅ Processing complete! {len(seen_seqs)} unique sequences have been written to: {output_csv}")



fa_folders = [
    f'{project_dir}/test/4dn4_ccl2_fa/seqs',
]

direct_fasta_to_csv(
    input_dirs=fa_folders, 
    output_csv=f'{project_dir}/test/4dn4_ccl2_fa/final_result.csv', 
    suffix=".pdb"
)

## run in a tmux terminal
```
bash demo_scripts/flowpacker_af3score.sh\
    test/4dn4_ccl2/filtered_links \
    test/4dn4_ccl2_fa/final_result.csv \
    test/af3score \
    1
```

In [ ]:
import pandas as pd
import os,shutil
df=pd.read_csv(f'{project_dir}/test/af3score/af3score_base_outputs/af3score_metrics.csv')
df['pdb_path']='test/af3score/flowpacker/flowpacker_outputs/run_1/'+df['description']+'.pdb'
df_f=df[df['iptm'] > 0.2]
print(df_f.shape)

out_dir = f'{project_dir}/test/af3score/filtered_links'
os.makedirs(out_dir, exist_ok=True)

df_f.apply(lambda row: os.link(os.path.abspath(row['pdb_path']), 
                                  os.path.join(out_dir, row['description']+'.pdb')), axis=1)

print(f"Linked {len(df_f)} files to {out_dir}")

# Step.3 Rosetta interface analysis

```
nohup bash demo_scripts/interface_analysis/0-run_rosetta.sh test/af3score/filtered_links test/iptm_filter_rst/ >> nohup.out &
```


In [26]:
!python demo_scripts/interface_analysis/get_interface_energy.py \
    --input_pdbdir test/af3score/filtered_links \
    --rosetta_dir test/iptm_filter_rst \
    --binder_id A \
    --target_id M \
    --output_dir test/iptm_filter_rst

Found pdb: 9
start Pool
  0%|                                                     | 0/9 [00:00<?, ?it/s]test/iptm_filter_rst/4dn4_ccl2_1_8.png
test/iptm_filter_rst/4dn4_ccl2_0_7.png
test/iptm_filter_rst/4dn4_ccl2_8_4.png
test/iptm_filter_rst/4dn4_ccl2_0_4.png
test/iptm_filter_rst/4dn4_ccl2_3_6.png
test/iptm_filter_rst/4dn4_ccl2_0_5.png
test/iptm_filter_rst/4dn4_ccl2_0_2.png
test/iptm_filter_rst/4dn4_ccl2_1_4.png
test/iptm_filter_rst/4dn4_ccl2_1_5.png
  binder_id target_id binder_res target_res  total  in_interface
0         A         M         52         22 -0.194          True
1         A         M         53         20 -2.314          True
Index(['binder_id', 'target_id', 'binder_res', 'target_res', 'total',
       'in_interface'],
      dtype='object')
  binder_id target_id binder_res target_res  total  in_interface
0         A         M          1         57 -0.141          True
1         A         M          2         57 -0.249          True
  binder_id target_id binder_res target

In [ ]:
# extract v < -5
import ast
residue_energy = pd.read_csv("test/iptm_filter_rst/residue_energy.csv")
residue_energy["fixed_residue"] = residue_energy["binder_energy"].apply(lambda x: "_".join([str(k) for k, v in ast.literal_eval(x).items() if v < -5]))
residue_energy["fixed_residue"] = residue_energy["fixed_residue"].astype(str)
residue_energy["num_fixed_residue"] = residue_energy["fixed_residue"].apply(
    lambda x: 0 if x == "" else len(x.split("_"))
)
print(residue_energy.shape)
residue_energy.to_csv('test/rst_intface_residue_energy_with_key_contacts.csv',index=False)

(9, 8)


# Step.4 Affinity maturation

## Step.4.1 Parameter Preparation

We need the **CDR regions**, the **framework**, and the **residue indices of key contacts** from the previous step with `v < -5`.


In [33]:
!python demo_scripts/pdb2fa.py test/af3score/filtered_links csv test/af3score/filtered_links.csv

seq saved at test/af3score/filtered_links.csv


In [39]:
# extract 
BASE='test'
residue_energy = pd.read_csv(f"{BASE}/rst_intface_residue_energy_with_key_contacts.csv")
seq = f"{BASE}/af3score/filtered_links.csv"
save_csv=f"{BASE}/fixed_residue.csv"

seq_df_multi = pd.read_csv(seq)
seq_df_multi['pdb_name']=seq_df_multi['pdb_name'].str.replace('.pdb','')
residue_energy = residue_energy.rename(columns={'pdbname':'pdb_name'})

residue_energy = pd.merge(residue_energy, seq_df_multi, on="pdb_name")
residue_energy["structure"] = residue_energy["pdb_name"].apply(lambda x: '_'.join(x.split("_")[:-1]))
residue_energy["num_fixed_residue"] = residue_energy["fixed_residue"].apply(lambda x: len(x.split("_")) if not pd.isna(x) else 0)

all_cand = []
for i, group in residue_energy.groupby("structure"):
    key_res = {}
    for j, row in group.iterrows():

        inter_res = {str(k):[v, row["sequence"][int(k)-1]] for k, v in ast.literal_eval(row["binder_energy"]).items() if v < -5}

        for k in inter_res:
            if k not in key_res:
                key_res[k] = inter_res[k]
            elif inter_res[k][0] <= key_res[k][0]:
                key_res[k] = inter_res[k]


    all_cand.append([row["pdbpath"], "_".join(row["pdb_name"].split('_')[:-1]), row["target_id"], row["binder_id"], key_res, len(key_res)])

        # key_res = {k: inter_res[k] if inter_res[k][0] <= key_res[k][0] else key_res[k] for k in inter_res}
all_cand_df = pd.DataFrame(all_cand, columns=["pdb_path", "pdb_name", "target_id", "binder_id", "key_res", "num_fixed_residues"])
print(all_cand_df.shape)
all_cand_df.to_csv(f'{save_csv}',index=False)

(4, 6)


In [ ]:
# merge key residues into one pdb file
import pandas as pd
import os
import ast
from tqdm import tqdm

def process_pdb_mutation_and_renumber(csv, pdb_output_dir, 
                                      renumber_chain='B',
                                      start_index=1):
    one_to_three = {
        'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS',
        'Q': 'GLN', 'E': 'GLU', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
        'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO',
        'S': 'SER', 'T': 'THR', 'W': 'TRP', 'Y': 'TYR', 'V': 'VAL'
    }
    
    df = pd.read_csv(csv)
    os.makedirs(pdb_output_dir, exist_ok=True)

    print(f"Start processing: Renaming Chain A & Renumbering Chain {renumber_chain} (start from {start_index})...")

    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        
        if not os.path.exists(row["pdb_path"]):
            print(f"Warning: {row['pdb_path']} not found, skipping...")
            continue

        resi_to_resname = {}
        if "key_res" in row and pd.notna(row["key_res"]):
            try:
                for resi, aa in ast.literal_eval(row["key_res"]).items():
                    resi_to_resname[int(resi)] = one_to_three.get(aa[1], "UNK")
            except Exception as e:
                print(f"Error parsing key_res for {row['pdb_name']}: {e}")

        with open(row["pdb_path"], "r") as f:
            pdb_lines = f.readlines()

        new_lines = []
        
        current_renumber_idx = start_index - 1
        last_seen_resi_id = None

        for line in pdb_lines:
            if not line.startswith("ATOM"):
                new_lines.append(line)
                continue

            chain_id = line[21]
            

            if chain_id == "A":
                resi = int(line[22:26])
                if resi in resi_to_resname:
                    new_resname = resi_to_resname[resi]

                    line = line[:17] + new_resname.ljust(3) + line[20:]

            elif chain_id == renumber_chain:
                current_resi_id_str = line[22:27]                
                if current_resi_id_str != last_seen_resi_id:
                    current_renumber_idx += 1
                    last_seen_resi_id = current_resi_id_str

                new_resi_str = f"{current_renumber_idx:>4}"
                line = line[:22] + new_resi_str + " " + line[27:]
            new_lines.append(line)
        output_pdb = os.path.join(pdb_output_dir, os.path.basename(row["pdb_name"])+".pdb")
        with open(output_pdb, "w") as f:
            f.writelines(new_lines)
    print("All done!")

process_pdb_mutation_and_renumber("test/fixed_residue.csv", "test/merge_motif_pdb/", 
    renumber_chain='M', start_index=9) # The residue numbering of the target chain is consistent with the input target file

Start processing: Renaming Chain A & Renumbering Chain M (start from 9)...


100%|██████████| 4/4 [00:01<00:00,  3.09it/s]

All done!


In [69]:
!python demo_scripts/get_cdr.py test/merge_motif_pdb test/filter_cdr_idx.csv

Starting processing 4 files with 110 cores...
Extracting Framework and CDR indices for Chain A...
100%|████████████████████████████████████████████| 4/4 [00:00<00:00,  5.12pdb/s]

Saved at test/filter_cdr_idx.csv


In [ ]:
import os
import ast
import pandas as pd

def safe_extract_keys(val):

    if pd.isna(val) or str(val).strip() == "":
        return ""
    try:
        d = ast.literal_eval(val)
        if isinstance(d, dict):
            return " ".join([str(k).strip() for k in d.keys()])
    except (ValueError, SyntaxError):
        return ""
    return ""

def process_indices(val, output_format="space"):

    if pd.isna(val) or str(val).strip() == "":
        return ""
    
    try:
        unique_sorted_ints = sorted(list(set(int(i) for i in str(val).split() if i.strip())))
        
        if output_format == "pdb":
            return ",".join([f"A{i}" for i in unique_sorted_ints])
        else:
            return " ".join([str(i) for i in unique_sorted_ints])
            
    except ValueError:
        return ""
# ==========================================
# 1. Load & Preprocess fw_csv
fw_csv = pd.read_csv('test/filter_cdr_idx.csv')
fw_csv = fw_csv[['pdb_name', 'fw_index', 'r2_cdr_pos']].copy()
fw_csv.rename(columns={'pdb_name': 'pdb_struc_name'}, inplace=True)

# 2. Load & Preprocess key_res_csv
key_res_csv = pd.read_csv("test/fixed_residue.csv")
output_path = "test/fixed_residue_setting_for_partial_flow.csv"

key_res_csv["pdb_struc_name"] = key_res_csv["pdb_name"]#.apply(lambda x: '_'.join(str(x).split("_")[:-1]))

# 3. Merge DataFrames

fixed_positions_df = fw_csv.merge(key_res_csv, on="pdb_struc_name", how='inner')

# 4. Extract Key Residues
fixed_positions_df["key_res_index"] = fixed_positions_df["key_res"].apply(safe_extract_keys)

# 5. Combine Indices (Raw String Concatenation)

raw_combined_indices = (
    fixed_positions_df['key_res_index'].fillna("").astype(str) + 
    " " + 
    fixed_positions_df['fw_index'].fillna("").astype(str)
)

# 6. Generate Final Columns

fixed_positions_df['r2_fixed_sequence_pm'] = raw_combined_indices.apply(lambda x: process_indices(x, output_format="space"))

fixed_positions_df['r2_fixed_sequence'] = raw_combined_indices.apply(lambda x: process_indices(x, output_format="pdb"))

fixed_positions_df['r2_fixed_structure'] = fixed_positions_df['key_res_index'].apply(lambda x: process_indices(x, output_format="pdb"))

os.makedirs(os.path.dirname(output_path), exist_ok=True)
fixed_positions_df.to_csv(output_path, index=False)

print(f"Data processed and saved to {output_path}")
print(fixed_positions_df[['r2_fixed_sequence_pm', 'r2_fixed_sequence']].head())

Data processed and saved to test/fixed_residue_setting_for_partial_flow.csv
                                r2_fixed_sequence_pm  \
0  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...   
1  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...   
2  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...   
3  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...   

                                   r2_fixed_sequence  
0  A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14...  
1  A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14...  
2  A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14...  
3  A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14...  


## Step.4.2 Partial Flow

In [ ]:
BASE='test'
input_dir = f"{BASE}/merge_motif_pdb"
output_dir = f"{BASE}/pf"
hotspot="M28,M39,M55,M61"
shelldir = f"{output_dir}/submit"
logdir = f'{output_dir}/log'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(shelldir, exist_ok=True)
os.makedirs(logdir, exist_ok=True)

binder_chain, target_chain = ["A","C"]
samples_per_target=8
t_start, t_end = [0.6,0.6]
interface_dist = 10
motif_contig = ""
model_weights = "path_to_ckpt"

fixed_positions_df = pd.read_csv(f"{BASE}/fixed_residue_setting_for_partial_flow.csv")
for i, row in fixed_positions_df.iterrows():

    pdb = os.path.join(input_dir, row["pdb_name"]+".pdb")
    pdb_name = row["pdb_name"]
    fixed_structure = row["r2_fixed_structure"]
    fixed_sequence = row["r2_fixed_sequence"]
    cdr_pos = row["r2_cdr_pos"]

    output_dir_temp = os.path.join(output_dir,row["pdb_name"])

    if os.path.exists(os.path.join(output_dir,pdb_name)):
        continue
    with open(f'{shelldir}/get_batch_{pdb_name}.sh', 'w') as f:
        f.write(f"#!/bin/bash \n"
                f"#SBATCH -p gpu41,gpu43 \n"
                f"#SBATCH --gres=gpu:1 \n"
                f"#SBATCH --ntasks=1 \n"
                f"#SBATCH --cpus-per-task=4\n"
                f"#SBATCH -N 1 \n"
                f"#SBATCH -o {logdir}/%j-{pdb_name}.out\n"
                f"#SBATCH -J {i}_partial\n"
                f"\n"
                )
        f.write(
            'echo "Job started at: $(date)"\n'
            'start_time=$(date +%s)\n'
            'export LAYERNORM_TYPE=fast_layernorm \n'
            'export CUTLASS_PATH=cutlass \n'
            'export MPLBACKEND=Agg \n'
        )
        cmd = ("miniconda3/envs/fm56/bin/python sample_antibody_nanobody_partial.py "
                f"--complex_pdb {pdb} "
                f"--start_t {t_start} "
                f"--fix_structure {fixed_structure} "
                f"--fix_sequence {fixed_sequence} "
                f"--antigen_chain M "
                f"--heavy_chain A "
                f"--config configs/inference_nanobody.yaml "
                f"--samples_per_target {samples_per_target} "
                f"--cdr_position {cdr_pos} "
                f"--specified_hotspots '{hotspot}' "
                f"--retry_Limit 10 "
                f"--model_weights \"{model_weights}\" "
                f"--output_dir \"{output_dir_temp}\" " 
                f"--name {pdb_name}"
            )

        f.write(cmd)
        f.write("\n")
        f.write(
            'end_time=$(date +%s)\necho "Job ended at: $(date)"\nelapsed=$((end_time - start_time))\necho "Elapsed time: $elapsed seconds"')
        
    cmd = f"sbatch {shelldir}/get_batch_{pdb_name}.sh"
    status, jobnum = subprocess.getstatusoutput(cmd)
    print(f"status={status}, jobnum:{jobnum}")

status=0, jobnum:Submitted batch job 2910935
status=0, jobnum:Submitted batch job 2910936
status=0, jobnum:Submitted batch job 2910937
status=0, jobnum:Submitted batch job 2910938


In [ ]:
import os
from pathlib import Path

src_root = Path("test/pf")
dst_dir = src_root / "link_samples"
dst_dir.mkdir(exist_ok=True)

for subdir in src_root.iterdir():
    if not subdir.is_dir():
        continue
    if subdir.name == "link_samples":
        continue

    for pdb in subdir.glob("sample*.pdb"):

        idx = pdb.stem.replace("sample", "")

        new_name = f"{subdir.name}_{idx}.pdb"
        dst_path = dst_dir / new_name
        if not dst_path.exists():
            os.symlink(pdb.resolve(), dst_path)


## Step.4.3 Sequence design

In [ ]:
# abmpnn 
input_dir = "test/pf"
output_dir = "test/fa"
resi = pd.read_csv('test/fixed_residue_setting_for_partial_flow.csv')
PREFIX='ccl2'
shelldir = f"{output_dir}/submit"
logdir = f'{output_dir}/logs'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(shelldir, exist_ok=True)
os.makedirs(logdir, exist_ok=True)

# Create the softlink directory if it doesn't exist
num_seq_per_target=4
sampling_temp=0.1
batch_size=num_seq_per_target
omit_AAs="C"
chains_to_design="A"
for i, folder in enumerate(os.listdir(input_dir)):
    output_dir_temp = os.path.join(output_dir, folder)
    if PREFIX not in folder or os.path.exists(output_dir_temp):
        continue
    else:
        print(folder)
        # pdb_name = "_".join(folder.split("_")[:-1])
        pdb_name = folder
        folder_with_pdbs = os.path.join(input_dir, folder)
        os.makedirs(output_dir_temp, exist_ok=True)
        fixed_positions = resi.loc[resi['pdb_name'] == pdb_name, 'r2_fixed_sequence_pm'].values[0]

        with open(f'{shelldir}/get_batch_{pdb_name}.sh', 'w') as f:
            f.write(f"#!/bin/bash \n"
                    f"#SBATCH -p gpu41,gpu43 \n"
                    f"#SBATCH --gres=gpu:1 \n"
                    f"#SBATCH --ntasks=1 \n"
                    f"#SBATCH --cpus-per-task=1\n"
                    f"#SBATCH -N 1 \n"
                    f"#SBATCH -o {logdir}/%j-{pdb_name}.out\n"
                    f"#SBATCH -J {i}_proteinmpnn\n"
                    f"\n"
                    )
            f.write(
                'echo "Job started at: $(date)"\n'
                'start_time=$(date +%s)\n'
                'export WANDB_MODE=disabled\n'
                'source miniconda3/etc/profile.d/conda.sh \n'
                'conda activate mlfold \n'
            )

            cmd = ( f"folder_with_pdbs={folder_with_pdbs} \n"
                    f'fixed_positions="{fixed_positions}" \n'
                    f"output_dir={output_dir_temp} \n\n"
                    f"path_for_parsed_chains={os.path.join(output_dir_temp,'parsed_pdbs.jsonl')} \n"
                    f"path_for_assigned_chains={os.path.join(output_dir_temp,'assigned_pdbs.jsonl')} \n\n"
                    f"path_for_fixed_positions={os.path.join(output_dir_temp,'fixed_pdbs.jsonl')} \n\n"
                    f"python ProteinMPNN-main/helper_scripts/parse_multiple_chains.py --input_path={folder_with_pdbs} --output_path=$path_for_parsed_chains \n"
                    f"python ProteinMPNN-main/helper_scripts/assign_fixed_chains.py --input_path=$path_for_parsed_chains --output_path=$path_for_assigned_chains --chain_list {chains_to_design} \n\n"
                    f"python ProteinMPNN-main/helper_scripts/make_fixed_positions_dict.py --input_path=$path_for_parsed_chains --output_path=$path_for_fixed_positions --chain_list {chains_to_design} --position_list '{fixed_positions}' \n"
                    f"python ProteinMPNN-main/protein_mpnn_run.py \\\n"
                    f"--path_to_model_weights ProteinMPNN-main/model_weights \\\n"
                    f"--model_name 'abmpnn' \\\n"
                    f"--jsonl_path $path_for_parsed_chains \\\n"
                    f"--chain_id_jsonl $path_for_assigned_chains \\\n"
                    f"--fixed_positions_jsonl $path_for_fixed_positions \\\n"
                    f"--out_folder $output_dir \\\n"
                    f"--num_seq_per_target {num_seq_per_target} \\\n"
                    f"--sampling_temp {sampling_temp} \\\n"
                    f"--batch_size {batch_size} \\\n"
                    f"--use_soluble_model \\\n"
                    f"--omit_AAs {omit_AAs} \n\n"
                    )

            # print(cmd)
            f.write(cmd)
            f.write(
                'end_time=$(date +%s)\necho "Job ended at: $(date)"\nelapsed=$((end_time - start_time))\necho "Elapsed time: $elapsed seconds"')
                
        cmd = f"sbatch {shelldir}/get_batch_{pdb_name}.sh"
        status, jobnum = subprocess.getstatusoutput(cmd)
        print(f"status={status}, jobnum:{jobnum}")

4dn4_ccl2_1
status=0, jobnum:Submitted batch job 2910958
4dn4_ccl2_8
status=0, jobnum:Submitted batch job 2910959
4dn4_ccl2_3
status=0, jobnum:Submitted batch job 2910960
4dn4_ccl2_0
status=0, jobnum:Submitted batch job 2910961


# Step.5 Scoring & Filtering

In [70]:
import os
from pathlib import Path
from multiprocessing import Pool, cpu_count
from tqdm.auto import tqdm
import pandas as pd

def _parse_single_fasta(fasta_path: Path) -> list:

    records = []
    path_str = str(fasta_path)
    try:
        with fasta_path.open("r", encoding="utf-8") as fh:
            seq_idx = -1
            seq_buf = []

            for line in fh:
                if line.startswith(">"):
                    if seq_buf:
                        records.append([path_str, "".join(seq_buf), seq_idx])
                        seq_buf = []
                    seq_idx += 1
                else:

                    stripped_line = line.strip()
                    if stripped_line:
                        seq_buf.append(stripped_line)

            if seq_buf:
                records.append([path_str, "".join(seq_buf), seq_idx])
    
    except Exception as e:
        print(f"[WARN] Skipping file {fasta_path} due to error: {e}")
        return []

    return records

def read_all_sequences_parallel(input_dir: str, output_csv: str) -> pd.DataFrame:
    fasta_paths = list(Path(input_dir).rglob("*.fa"))
    if not fasta_paths:
        print("Warning: No .fa files found in the specified directory.")
        df = pd.DataFrame(columns=["fasta", "seq", "seq_idx"])
        df.to_csv(output_csv, index=False)
        return df
    
    print(f"Found {len(fasta_paths)} files. Starting parallel parsing...")

    num_processes = cpu_count()
    all_records = []

    with Pool(processes=num_processes) as pool:
        with tqdm(total=len(fasta_paths), desc="Parsing FASTA files in parallel") as pbar:
            for records_from_one_file in pool.imap_unordered(_parse_single_fasta, fasta_paths):
                if records_from_one_file:
                    all_records.extend(records_from_one_file)
                pbar.update(1)

    if not all_records:
        print("Warning: All files are empty or could not be parsed; the resulting CSV will be empty.")

    print("\nAll files have been parsed. Creating DataFrame and saving to CSV...")
    df = pd.DataFrame(all_records, columns=["fasta", "seq", "seq_idx"])
    df.to_csv(output_csv, index=False)
    print(f"Successfully saved {len(df)} sequences to {output_csv}")
    
    return df


input_directory = "test/fa"
output_csv = "test/fa.csv"
sequences_df = read_all_sequences_parallel(input_directory, output_csv)
sequences_df['seq_idx']= '' + sequences_df['seq_idx'].astype(str)
sequences_df.to_csv(output_csv,index=False)

Found 32 files. Starting parallel parsing...


Parsing FASTA files in parallel:   0%|          | 0/32 [00:00<?, ?it/s]


All files have been parsed. Creating DataFrame and saving to CSV...
Successfully saved 160 sequences to test/fa.csv


In [ ]:
df = pd.read_csv('test/fa.csv')
df = df[
    ~df["seq_idx"].isin(['0',0]) # remove wt
]
df['orig_path'] = df['fasta']
df['orig_path'] = (df['orig_path']
                   .str.replace('test', 'test', regex=False)
                   .str.replace('/fa', '/pf/linked_samples', regex=True)
                   .str.replace('/seqs/sample', '_', regex=False)
                   .str.replace('.fa', '.pdb', regex=False)
                  )

df['link_name'] = df['orig_path'].str.split('/').str[-1]
df = df[['link_name','seq','seq_idx']]
df.to_csv('test/pf_fa_sum.csv',index=False)
df

,link_name,seq,seq_idx
1,4dn4_ccl2_3_7.pdb,EVQLVESGGGLVQPGGSLRLSCAASGGSMSSYAMAWYRQAPGKGRE...,1
2,4dn4_ccl2_3_7.pdb,EVQLVESGGGLVQPGGSLRLSCAASGGSMSSYAMAWYRQAPGKGRE...,2
3,4dn4_ccl2_3_7.pdb,EVQLVESGGGLVQPGGSLRLSCAASGGTMSSYAMAWYRQAPGKGRE...,3
4,4dn4_ccl2_3_7.pdb,EVQLVESGGGLVQPGGSLRLSCAASGGSMSSYAMAWYRQAPGKGRE...,4
6,4dn4_ccl2_1_4.pdb,EVQLVESGGGLVQPGGSLRLSCAASGGNISSSYMAWYRQAPGKGRE...,1
...,...,...,...
154,4dn4_ccl2_1_7.pdb,EVQLVESGGGLVQPGGSLRLSCAASGGSVSDSTMAWYRQAPGKGRE...,4
156,4dn4_ccl2_1_5.pdb,EVQLVESGGGLVQPGGSLRLSCAASGGTFSSSSMAWYRQAPGKGRE...,1
157,4dn4_ccl2_1_5.pdb,EVQLVESGGGLVQPGGSLRLSCAASGGTFSSSSMAWYRQAPGKGRE...,2
158,4dn4_ccl2_1_5.pdb,EVQLVESGGGLVQPGGSLRLSCAASGGTFSSSSMAWYRQAPGKGRE...,3


In [ ]:
## run in a tmux terminal
```
bash demo_scripts/flowpacker_af3score.sh\
    test/pf/link_samples \
    test/pf_fa_sum.csv \
    test/af3score_r2 \
    3
```

In [ ]:
import pandas as pd
import os,shutil
df=pd.read_csv('test/af3score_r2/af3score_base_outputs/af3score_metrics.csv')
df['pdb_path']='test/af3score_r2/flowpacker/flowpacker_outputs/run_1/'+df['description']+'.pdb'
df_f=df[(df['iptm'] > 0.5)&(df['ptm_A'] > 0.8)]
print(df_f.shape)
out_dir = f'test/af3score_r2/filtered_links'
os.makedirs(out_dir, exist_ok=True)
df_f.apply(lambda row: os.link(os.path.abspath(row['pdb_path']), 
                                  os.path.join(out_dir, row['description']+'.pdb')), axis=1)
print(f"Linked {len(df_f)} files to {out_dir}")

#### Note for Testing
In some cases, you may find that **no designs pass the filter criteria**. 

To address this, you can **increase the number of sampled designs** or **adjust the hotspots**. 

A few pilot runs are recommended to optimize these settings.


# Steq.6 Rosetta

In [ ]:
!python demo_scripts/submit_relax_comp.py test/af3score_r2/filtered_links

In [ ]:
csv_path = 'test/af3score_r2/filtered_links/rst_itf_nofix'

all_data = pd.DataFrame()
csv_files = glob.glob(os.path.join(csv_path, 'rosetta_complex*.csv'))

for csv in csv_files:
    data = pd.read_csv(csv)
    all_data = pd.concat([all_data, data], ignore_index=True)

output_csv_path = os.path.join(f'test/all_rst_comp.csv') # interface score
all_data.to_csv(output_csv_path, index=False)

# Steq.7 AF3 Refolding

Refolding is performed **without a template**.  
We **recommend using 20 seeds × 5 models** to improve prediction quality.

## dockq

In [ ]:
!python demo_scripts/run_DockQv2_eachfolder_v2.py \
  --input_dir path_to_af3_out \
  --reference_dir test/af3score_r2/filtered_links \
  --output_dir test/dockq \
  --shell_dir test/dockq_shell

In [ ]:
!python demo_scripts/HNMT/parse_dockq_scores.py test/dockq

# Final Filtering Criteria

### Structural Quality Thresholds
The following filters are applied to retain high-confidence models:

- **DockQ > 0.49**
- **AF3 pTM > 0.8**
- **AF3 ipTM > 0.7**

### Ranking Score
Candidates are ranked using the following empirical score:
```Score = (AF3 ipTM × 100) − Interface Score```

### Notes
- This score is **only meaningful for ranking candidates within the same target and the same molecular type**.
- It does **not** have an absolute physical or biological interpretation.
- The purpose is **relative prioritization**, not cross-target comparison.

Good luck.

